<a href="https://colab.research.google.com/github/vatsaryan07/Workshop-Review-Sentiment-Analysis-Web-App/blob/main/Sentiment_Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
import random
import pickle
from statistics import mode
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from google.colab import files
import pandas as pd
import re
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
"""
To first access the data from the Kaggle website, we can either : 

a) Download it from kaggle and load it in manually


b) Connect Kaggle to Colab notebook and fetch the data



"""
# Using the kaggle dataset for IMDB review dataset
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle (1).json


In [ ]:
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!unzip imdb-dataset-of-50k-movie-reviews.zip

-rw-r--r-- 1 root root 67 Mar 20 11:33 kaggle.json
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  imdb-dataset-of-50k-movie-reviews.zip
replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: IMDB Dataset.csv        


In [ ]:

# Reading the Dataset
orig_data = pd.read_csv('IMDB Dataset.csv')

# Initializing the list for our bag of words
all_words =  []

# In POS tagging, adjectives denoted by "J", thus we specify it as an allowed word type
allowed_word_types = ["J"]

# Initializing a list for the IMDB Dataset
documents = []

# Fetching stop words from NLTK library and removing the word "not" from the list of stop words
stop_words = list(set(stopwords.words('english')))
stop_words.remove('not')


for p,q in zip(orig_data['review'],orig_data['sentiment']):
  # Appending the dataset to the list 
  documents.append((p,q))

  # Removing punctuations and other special characters, leaving behind only letters
  cleaned = re.sub(r'[^(a-zA-Z)\s]','',p)
  
  # Making all the words lowercase for homogenuity
  lowered = cleaned.lower()

  # Tokenizing the words to obtain list of words rather than full sentence
  tokenized = word_tokenize(lowered)

  # Removing any stopwords like "and" "the" 
  stopped = [w for w in tokenized if not w in stop_words]
  
  # POS tagging each word
  pos = nltk.pos_tag(stopped)

  for w in pos:
    if w[1][0] in allowed_word_types:
      # Checking if the word is adjective, only then appending it to the bag of words
      all_words.append(w[0].lower())


In [ ]:
# Making a Frequency Distribution to sort adjectives by their occurence
all_words = nltk.FreqDist(all_words)

# Keeping the 1000 most occuring adjectives
word_features = list(all_words.keys())[:1000]

# Grouping outliers/traps with our particular model in a dictionary
risk_words = {
    'not good' : 'bad',
    'not bad' : 'good',
    'not nice' : 'bad',
    'hate' : 'bad'
}



# Adding the list of risk_words to the word_feature list
for i in risk_words:
  word_features.append(i)

# Defining a function to find all features in a given line based on the BagOfWords (making a one-hot-encoding)
def find_features(word_features,document):
    """
    In this function, suppose we give an input as " The workshop was wonderful and interesting, but a little boring towards the end"
    The function will then return a 1004 size dictionary, where for each feature (adjective) in our BoW, we either have True, ie 
    the adjective is in the sentence, or False, meaning the adjective is not in the sentence

    Thus, 'wonderful', 'interesting' and 'boring' features will have true while the rest have false.
    """
    document = document.lower()
    words = word_tokenize(document)
    # Converting into set to remove duplicates to save time 
    document_words = set(words)
    features = {}
    for w in word_features:
        # w in document_words will either return True or False
        features['contains({})'.format(w)] = (w in document_words)
    for w in risk_words.keys():
        if w in document:
          features['contains({})'.format(risk_words[w])] = (w in document)
          if w != 'hate':
            # For a sentence containing 'not good' we make the 'good' feature False to avoid the trap of thinking good occurs in the sentence
            features['contains({})'.format(w.split()[1])] = False
    return features

def show_features(word_features,document):
    """
    Same functionality as find_features() used to display the features for better understanding
    """
    document = document.lower()
    words = word_tokenize(document)
    document_words = set(words)
    features = {}
    for w in word_features:
        features['contains({})'.format(w)] = (w in document_words)
    for w in risk_words.keys():
        if w in document:
          features['contains({})'.format(risk_words[w])] = (w in document)
          if w != 'hate':
            features['contains({})'.format(w.split()[1])] = False
    for w in features.keys():
      if features[w] == True:
        print(w)

temp = show_features(word_features,'The movie was not good')

# Making our training set as the features for every sentence in the IMDB Model with it's corresponding sentiment
featuresets = [(find_features(word_features,review),category) for (review,category) in documents]

# Shuffling it to prevent unnecessary biasing
random.shuffle(featuresets)

contains(bad)


In [ ]:
print(len(featuresets))

50000


In [ ]:
training_set = featuresets[:40000]
testing_set = featuresets[40000:]

# An inbuilt Naive Bayes Classifier is built to save time and effort on training 
classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 81.97


In [ ]:
def sentiment(classifier,text):
    feats = find_features(word_features,text)
    return classifier.classify(feats)#, classifier.confidence(feats)

sentiment(classifier,'The workshop was boring and dull, it was bad and I never want to attend it again')

'negative'

In [ ]:
import pickle
pickle.dump(classifier,open('SentiAnalysisfinal.sav','wb'))
files.download('SentiAnalysisfinal.sav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

with open("word_features_final.txt", "wb") as fp:
  pickle.dump(word_features, fp)
files.download("word_features_final.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.upload()
filename = 'SentiAnalysisfinal.sav'
with open(filename,'rb') as fp:
  loaded_model = pickle.load(fp)
  print(sentiment(loaded_model,'The workshop was good, very nice and intuitive'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

positive
